In [ ]:
# Cell 1: Environment Setup
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0, ResNet50
from tensorflow.keras import layers, models, utils
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import cv2
import matplotlib.pyplot as plt
import os
from scipy.ndimage import zoom
import time

In [ ]:
# Cell 2: Optimized Data Loading & Augmentation for Malignant Cases

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# Set paths
TRAIN_IMAGE_DIR = '/kaggle/input/isic-2024-challenge/train-image/image/'
AUG_IMAGE_DIR = '/kaggle/working/augmented-images/' 

# Create directory if not exists
os.makedirs(AUG_IMAGE_DIR, exist_ok=True)

# Augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,  
    zoom_range=0.1,  
    horizontal_flip=True,  
    brightness_range=[0.8, 1.2],  
    fill_mode='nearest'
)

def augment_malignant_images(malignant_df, target_count=20000):
    """Generates synthetic malignant images to balance dataset."""
    new_malignant_paths = []
    total_needed = target_count - len(malignant_df)
    augment_per_image = total_needed // len(malignant_df)  # ~50x per image
    
    print(f"Generating {total_needed} new malignant images...")
    
    for _, row in malignant_df.iterrows():
        img_path = row['image_path']
        img = load_img(img_path)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Generate augmented images
        for i in range(augment_per_image):
            aug_img = datagen.flow(img_array, batch_size=1)[0][0]
            aug_img = array_to_img(aug_img)
            
            # Save new image
            new_img_path = os.path.join(AUG_IMAGE_DIR, f"aug_{row['isic_id']}_{i}.jpg")
            aug_img.save(new_img_path)
            new_malignant_paths.append({'isic_id': f"aug_{row['isic_id']}_{i}", 'target': 1, 'image_path': new_img_path})
    
    print(f"Augmentation complete! {len(new_malignant_paths)} new malignant images created.")
    return pd.DataFrame(new_malignant_paths)

def load_balanced_data():
    start = time.time()
    
    # 1. Load Metadata
    print("Loading metadata...")
    train_meta = pd.read_csv(
        '/kaggle/input/isic-2024-challenge/train-metadata.csv',
        usecols=['isic_id', 'target'],
        dtype={'isic_id': 'string', 'target': 'int8'},
        low_memory=False
    )
    
    # 2. Build Image Paths
    print("Building paths...")
    train_meta['image_path'] = TRAIN_IMAGE_DIR + train_meta['isic_id'] + '.jpg'
    
    # 3. Filter Existing Images
    print("Verifying files...")
    existing_files = set(os.listdir(TRAIN_IMAGE_DIR))
    mask = (train_meta['isic_id'] + '.jpg').isin(existing_files)
    train_df = train_meta[mask].copy()
    
    # 4. Split Malignant and Benign
    print("Balancing classes...")
    malignant_df = train_df[train_df['target'] == 1].copy()
    benign_df = train_df[train_df['target'] == 0].copy()
    
    # 5. Augment Malignant Images
    aug_malignant_df = augment_malignant_images(malignant_df, target_count=20000)
    
    # 6. Sample 20,000 Benign Images
    benign_sampled = benign_df.sample(n=20000, random_state=42)
    
    # 7. Merge Augmented Malignant + Benign
    final_df = pd.concat([aug_malignant_df, benign_sampled], ignore_index=True)
    
    # 8. Shuffle Data
    final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Data loaded & augmented in {time.time()-start:.2f}s")
    return final_df

# Run data loading & augmentation
train_df = load_balanced_data()

print("\nFinal class distribution:")
print(train_df['target'].value_counts())


In [ ]:
# Cell 3: Data Preprocessing & Splitting
# Split data
train_data, val_data = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['target'],
    random_state=42
)

# Preprocessing function
def preprocess_image(path, img_size=(256, 256)):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img / 255.0

In [ ]:
# Cell 4: 2D Ensemble Model
def build_2d_ensemble():
    input_layer = layers.Input(shape=(256, 256, 3))
    
    # Feature Extraction
    effnet = EfficientNetV2B0(include_top=False, weights='imagenet')(input_layer)
    resnet = ResNet50(include_top=False, weights='imagenet')(input_layer)
    
    # Feature Processing
    x1 = layers.GlobalAveragePooling2D()(effnet)
    x2 = layers.GlobalAveragePooling2D()(resnet)
    merged = layers.concatenate([x1, x2])
    
    # Classifier
    x = layers.Dense(512, activation='relu')(merged)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

In [ ]:
# Cell 5: 2D Model Training
# Data generators
train_gen = tf.data.Dataset.from_tensor_slices(
    (train_data['image_path'].values, train_data['target'].values)
).map(lambda x,y: (preprocess_image(x), y)).batch(32)

val_gen = tf.data.Dataset.from_tensor_slices(
    (val_data['image_path'].values, val_data['target'].values)
).map(lambda x,y: (preprocess_image(x), y)).batch(32)

# Train and save
model_2d = build_2d_ensemble()
history_2d = model_2d.fit(train_gen, validation_data=val_gen, epochs=15)
model_2d.save('/kaggle/working/2d_model.h5')

# After saving 2D model
print("Files in working directory:")
print(os.listdir('/kaggle/working/'))

# Verify model file exists
if os.path.exists('/kaggle/working/2d_model.h5'):
    print("\n2D model saved successfully!")
    print(f"File size: {os.path.getsize('/kaggle/working/2d_model.h5')/1e6:.1f} MB")
else:
    print("\nError: 2D model not saved!")

# Plot accuracy
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(history_2d.history['accuracy'], label='Train')
plt.plot(history_2d.history['val_accuracy'], label='Validation')
plt.title('2D Model Accuracy')
plt.legend()

plt.subplot(122)
plt.plot(history_2d.history['auc'], label='Train')
plt.plot(history_2d.history['val_auc'], label='Validation')
plt.title('2D Model AUC')
plt.legend()
plt.show()



In [ ]:
# Cell 6: Monodepth 3D Augmentation with Visualization
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import marching_cubes

class VolumetricConverter:
    def __init__(self, depth=16, img_size=(256, 256)):
        self.depth = depth
        self.img_size = img_size
        self.aug = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='constant'
        )

    def convert_to_3d(self, img_array):
        """Convert a 2D image into a 3D volumetric structure"""
        slices = []
        base_img = img_array.numpy() if tf.is_tensor(img_array) else img_array
        for i in range(self.depth):
            augmented = self.aug.random_transform(base_img)
            slices.append(augmented[:, :, 0])  # Use grayscale intensity for volume

        volume = np.stack(slices, axis=2)  # Shape: (256, 256, depth)
        return volume

def visualize_3d_and_2d(volume_3d, original_2d):
    """Render 3D volumetric structure and its 2D counterpart"""
    fig = plt.figure(figsize=(10, 12))

    # 3D Volumetric Rendering
    ax1 = fig.add_subplot(2, 1, 1, projection='3d')
    verts, faces, _, _ = marching_cubes(volume_3d, level=np.mean(volume_3d))
    ax1.plot_trisurf(verts[:, 0], verts[:, 1], faces, verts[:, 2], cmap='coolwarm', alpha=0.7)
    ax1.set_title("3D Volumetric Representation")

    # 2D Image Display
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.imshow(original_2d, cmap='gray')
    ax2.set_title("Original 2D Image")
    ax2.axis('off')

    plt.show()

# Initialize converter
converter = VolumetricConverter(depth=16)

# Convert entire dataset to 3D
df_3d = []
for idx, row in train_df.iterrows():
    img_path = row['image_path']
    original_img = preprocess_image(img_path)  # Load and preprocess 2D image
    volume_3d = converter.convert_to_3d(original_img)  # Convert to 3D
    df_3d.append({'image_path': img_path, 'volume_3d': volume_3d})  # Store 3D version

# Convert to DataFrame
df_3d = pd.DataFrame(df_3d)

# Select a random image from df_3d for visualization
sample_idx = np.random.choice(len(df_3d))
sample_data = df_3d.iloc[sample_idx]
sample_3d = sample_data['volume_3d']
sample_2d = preprocess_image(sample_data['image_path'])

# Visualize both 3D and 2D versions
visualize_3d_and_2d(sample_3d, sample_2d)


In [ ]:
# Cell 7: 3D CNN Training & Evaluation
# 1. Prepare 3D dataset
# Convert DataFrame to numpy arrays
X = np.array(df_3d['volume_3d'].tolist())[..., np.newaxis] 
y = train_df['target'].values

# Verify input shape
print(f"Input shape: {X.shape}") 

# 2. Split into train/validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,
    random_state=42
)

# 3. Build 3D CNN model
def build_3d_cnn(input_shape=(256, 256, 16, 1)):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # Feature extraction
        layers.Conv3D(32, (3, 3, 3), activation='relu'),
        layers.MaxPooling3D((2, 2, 2)),
        layers.BatchNormalization(),
        
        layers.Conv3D(64, (3, 3, 3), activation='relu'),
        layers.MaxPooling3D((2, 2, 2)),
        layers.BatchNormalization(),
        
        # Classification
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

# 4. Train the model
model_3d = build_3d_cnn()
model_3d.summary()

history = model_3d.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=8,
    verbose=1
)

# 5. Save the model
model_3d.save('/kaggle/working/3d_cnn_model.h5')
print("Model saved successfully!")

# 6. Visualize training history
plt.figure(figsize=(15, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Check for model files
print("Files in working directory:")
print(os.listdir('/kaggle/working/'))

# Verify specific models
print("\n2D model exists:", os.path.exists('/kaggle/working/2d_model.h5'))
print("3D model exists:", os.path.exists('/kaggle/working/3d_model.h5'))

In [ ]:
# Cell 8: Prediction Pipeline
# 1. Load models
model_2d = tf.keras.models.load_model('/kaggle/working/2d_model.h5')
model_3d = tf.keras.models.load_model('/kaggle/working/3d_model.h5')

# 2. Preprocess test image
def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (256, 256))
    return img / 255.0

test_image = preprocess_image('/kaggle/input/skin-cancer-test/basal-cell-carcinoma-stock.jpg')

# 3. 2D Prediction
planar_pred = model_2d.predict(np.expand_dims(test_image, 0))[0][0]

# 4. 3D Conversion (Fixed)
class VolumetricConverter:
    def __init__(self, depth=16):
        self.depth = 16
        self.aug = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True
        )
    
    def convert_to_3d(self, img_array):
        """Convert to (256, 256, 16, 1) grayscale volume"""
        # Convert to grayscale
        gray_img = tf.image.rgb_to_grayscale(img_array)
        slices = []
        for _ in range(self.depth):
            augmented = self.aug.random_transform(gray_img.numpy())
            slices.append(augmented[..., 0])  # Remove channel dim
        return np.stack(slices, axis=2)[..., np.newaxis]  # Add single channel

converter = VolumetricConverter()
test_volume = converter.convert_to_3d(test_image)
test_volume = np.expand_dims(test_volume, axis=0)  # Add batch dimension

# 5. 3D Prediction
depth_pred = model_3d.predict(test_volume)[0][0]

# 6. Format and display results
def format_pred(pred):
    return f"Malignant: {pred*100:.1f}%\nBenign: {(1-pred)*100:.1f}%"

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(test_image)
plt.title("Test Image")

plt.subplot(1, 2, 2)
plt.text(0.1, 0.7, "Planar Analysis (2D):\n" + format_pred(planar_pred), 
         fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
plt.text(0.1, 0.4, "Depth Analysis (3D):\n" + format_pred(depth_pred),
         fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
plt.axis('off')
plt.show()

In [ ]:
# Cell 9: Explainable AI and Heatmap Visualization

# 1. Function to Compute Grad-CAM Heatmap
def get_gradcam_heatmap(model, img_array, layer):
    """Generates Grad-CAM heatmap for 2D models."""
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.expand_dims(img_array, axis=0))
        loss = predictions[:, 0]  

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = np.maximum(heatmap, 0)  
    heatmap /= np.max(heatmap) 
    return heatmap

# 2. Extracting the last conv layers from EfficientNet and ResNet
effnet = model_2d.get_layer("efficientnetv2-b0")
resnet = model_2d.get_layer("resnet50")

effnet_last_conv = effnet.layers[-2]  
resnet_last_conv = resnet.layers[-2]  

# 3. Compute Heatmaps for 2D Model
heatmap_2d_effnet = get_gradcam_heatmap(model_2d, test_image, layer=effnet_last_conv)
heatmap_2d_resnet = get_gradcam_heatmap(model_2d, test_image, layer=resnet_last_conv)

# Merge both heatmaps (ensemble Grad-CAM)
heatmap_2d = (heatmap_2d_effnet + heatmap_2d_resnet) / 2
